In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
pages=25

In [5]:
driver = webdriver.Chrome()
output_file = open("output.txt", "w",encoding='utf-8')  

for page in range(pages):
    driver.get(f"https://www.animecharactersdatabase.com/episodetranscript.php?pid=104802&epid={page+1}")
    transcript = driver.find_element(By.ID, 'transcript').text
    output_file.write(transcript)

output_file.close()  
driver.quit()
